In [2]:
import numpy as np

def read_npz_file(file_path):
    """
    Function to read data from a .npz file.

    Parameters:
        file_path (str): Path to the .npz file.

    Returns:
        data (dict): A dictionary containing data stored in the .npz file.
    """
    data = np.load(file_path)
    return data

In [5]:
file_path = 'tiny_nerf_data.npz'
data_dict = read_npz_file(file_path)

In [8]:
data_dict